In [1]:
import tkinter as tk
from tkinter import messagebox
import requests
from PIL import Image, ImageTk
import io

API_KEY = 'b8c742322b5848cf84f143312242505'
BASE_URL = 'https://api.weatherapi.com/v1/current.json'

def get_weather_data(location):
    complete_url = f"{BASE_URL}?key={API_KEY}&q={location}"
    response = requests.get(complete_url)
    if response.status_code == 200:
        return response.json()
    else:
        return None
class WeatherApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Weather App")
        self.root.geometry("400x400")
        
        self.city_label = tk.Label(root, text="Enter City Name or Zip Code:")
        self.city_label.pack(pady=10)
        
        self.city_entry = tk.Entry(root)
        self.city_entry.pack(pady=5)
        
        self.search_button = tk.Button(root, text="Search", command=self.search_weather)
        self.search_button.pack(pady=5)
        
        self.result_label = tk.Label(root, text="", font=("Helvetica", 12))
        self.result_label.pack(pady=20)
        
        self.weather_icon_label = tk.Label(root, image=None)
        self.weather_icon_label.pack(pady=10)
    
    def search_weather(self):
        city = self.city_entry.get()
        if city:
            weather_data = get_weather_data(city)
            if weather_data and 'current' in weather_data:
                self.display_weather(weather_data)
            else:
                messagebox.showerror("Error", "City not found or invalid API key!")
        else:
            messagebox.showerror("Error", "Please enter a city name or zip code")
    
    def display_weather(self, data):
        location = data['location']
        current = data['current']
        
        city = location['name']
        country = location['country']
        temp_c = current['temp_c']
        condition = current['condition']['text']
        icon_url = f"http:{current['condition']['icon']}"
        
        weather_info = f"City: {city}, {country}\nTemperature: {temp_c}°C\nWeather: {condition}"
        self.result_label.config(text=weather_info)
        
        # Fetch and display the weather icon
        icon_response = requests.get(icon_url, stream=True)
        if icon_response.status_code == 200:
            icon_image = Image.open(io.BytesIO(icon_response.content))
            icon_photo = ImageTk.PhotoImage(icon_image)
            self.weather_icon_label.config(image=icon_photo)
            self.weather_icon_label.image = icon_photo
        else:
            self.weather_icon_label.config(image=None)
if __name__ == "__main__":
    root = tk.Tk()
    app = WeatherApp(root)
    root.mainloop()
